<table border=1 width=100%>
    <tr><td style="border: 1px solid black; width:600px; height:40px; text-align: center;"><font size=4 color=blue><b>[4차시] 학습목표</b></font></td></tr>       
    <tr><td style="border: 1px solid black; text-align: left;"><font size=3>
        
○ 기본적인 Flask 사용법 학습하기<br>
○ 웹 화면에 카메라 영상 출력하기<br>
○ 웹 출력 영상을 Gray 이미지로 출력하기
        
</font></td></tr>   
</table>

- 파이썬은 기본적으로 동기방식으로 작동한다.
- 플라스크도 동기방식으로 작동한다.
- 순서대로 실행되는 구조라는 뜻이다.

<table border=1 width=100%>
    <tr><td style="border: 1px solid black; width:600px; height:40px; text-align: center;"><font size=4 color=blue><b>[4차시] 학습요약</b></font></td></tr>       
    <tr><td style="border: 1px solid black; text-align: left;"><font size=3>
        
○ <font color=red>ret, buffer = cv2.imencode()</font> : 영상 인코딩 함수<br>
○ yield() : 반복 실행 중에 중간 과정을 반환할 때 사용<br><br>

○ Flask : 파이썬으로 제작된 웹 프레임워크 (DJango의 축소 버전)<br>
○ <font color=red>app = Flask(__name__)</font> : Flask 객체를 app 변수에 할당<br>
○ <font color=red>@app.route("/")</font> : Flask에게 어떤 URL이 해당 함수를 실행하는지 알려줌<br>
○ <font color=red>@app.route("/hello")</font> : 서버 주소에 추가적인 URL을 설정<br>
○ <font color=red>@app.route("/hello/<value>")</font> : URL에 value 값을 설정<br>
○ <font color=red>app.run(host='127.0.0.1', port=5000)</font> : 서버 IP, 포트 등을 설정하고 서버를 실행<br><br>

○ <font color=red>render_template("hello.html")</font> : html 문서를 반환하기<br>
○ <font color=red>Response(stream_with_context())</font> : 데이터 스트리밍을 반환하기
        
</font></td></tr>   
</table>

# Flask 설치 및 서버 실행하기

- 플라스크(Flask)는 파이썬으로 작성된 마이크로 웹 프레임워크의 하나
  - 특별한 도구나 라이브러리가 필요 없음

In [ ]:
# Flask 설치
!pip install flask

## 데이터 스트리밍 구현

###구현 방법

- <font color=red>Response(stream_with_context(test()))</font>
  - 텍스트 스트리밍 함수 test()를 실행
  
  <img src="./data/lecture_image/04_flask05.png" width=50%>  

In [ ]:
def test():
    yield "Hello"  #yieldsms return과 비슷한 함수
    yield "World"
    yield "^^"  #return 은 반환하고 돌아가는데 yield는 return하고 나가지 않고 함수 안에 머뭄
                 #함수를 실행하다 중간 결과 값을 받고 싶을 때 유용하다.
    
for i in test():
    print(i)

In [ ]:
import time

def return_abc():
    temp = []
    
    #ABC 문장을 1초마다 1그자씩 읽어서 리스트를 저장 
    for ch in "ABC":
        time.sleep(1)
        temp.append(ch)
    return temp  #데이터가 다 저장된 뒤 마지막 결과물 반환

for i in return_abc():
    print(i)


In [ ]:
def yield_abc():
    temp = []
    
    #ABC 문장을 1초마다 1그자씩 읽어서 리스트를 저장 
    for ch in "ABC":
        time.sleep(1)
        temp.append(ch)
        yield ch  #데이터가 다 저장된 뒤 마지막 결과물 반환
    
for i in yield_abc():
    print(i)    

- yield() 함수 배우기

  
  <img src="./data/lecture_image/04_yield.png" width=70%>  

## 웹 브라우저에 카메라 영상 출력하기

- <font color=red>ret, buffer = cv2.imencode('.jpg', frame)</font> : frame 이미지를 jpg로 인코딩

- <font color=red>frame = buffer.tobytes()</font> : 전송을 위해 인코딩된 이미지를 byte 형식으로 변환


- <font color=red>yield (b'--frame\r\n' b'Content-Type: image/jpeg\r\n\r\n' + frame + b'\r\n') </font>

  - b : byte 형식임을 의미
  - --frame : 프레임을 표시
  - Content-Type: image/jpeg : 문서가 jpg 이미지임을 표시

In [ ]:
import cv2

cap = cv2.VideoCapture(0)

#이미지 읽어서 한 프레임씩 바로 전송하는 스트리밍 함수 만들기
def get_frames():
    while True:
        ret, frame = cap.read()

        #스트리밍 서비스 이기 때문에 프레임 이미지를 읽지 못했다면 다음 프레임 이미지를 읽도록 하기 위해 컨티뉴~
        if not ret :
            continue
        else:
            ret1.buffer = cv2.imencode('.jpg', frame)
            #전송하기 위해서 바이트로 변환
            frame = buffer.tobytes()

            #전송
            yield (b'--frame\r\n' b'Content-Type: image/jpeg\r\n\r\n' + frame + b'\r\n')

- index.html를 작성하고 templates 폴더에 저장
  - video_feed : 실행할 함수명

In [ ]:
%%writefile ./templates/index.html 

<html>
<body>
<div class="container">
    <div class="row">
        <div class="col-lg-8  offset-lg-2">
            <h3 class="mt-5">Live Streaming</h3>
            <img src="{{ url_for('video_feed') }}" width="50%">
            <img src="{{ url_for('video_feed2') }}" width="50%">
        </div>
    </div>
</div>
</body>
</html>

- <font color=red>Response(get_frames(), mimetype='multipart/x-mixed-replace; boundary=frame')</font>

  - get_frames() : 호출할 함수명
  - mimetype : 클라이언트에게 전송된 문서의 타입을 알려주기 위한 파라미터 (type/subtype)
  - multipart : 복합문서 타입 (파일, 영상 등)을 의미
  - x-mixed-replace : x (추가적인 확장 형식), mixed (복합문서), repalce (subtype을 다음 메시지로 대체)
  - boundary : 복합문서 내의 각 문서들을 구분하는 분리자 (동영상이므로 frame으로 구분)

In [ ]:

#안되는 코드
from flask import Flask, render_template, Response

app = Flask(__name__)

#초기화면
@app.route("/")
def index():
    return render_template("index.html")  #render_template 함수 자체가 tamplates 폴더 안에서 찾는 함수임

@app.route('/video_feed')
def video_feed():
    return Response(get_frames(),
                   mimetype='multipart/x-mixed-replace; boundary=frame')

if __name__ == '__main__':
    app.run(host='192.168.70.90', port=5050)

In [ ]:
import cv2

cap = cv2.VideoCapture(0)
# 이미지를 읽어서 한프레임씩 바로 전송하는 스트리밍 함수
def get_frames() :
    while True :
        ret, frame = cap.read()
        # 현재 프레임이미지를 읽지 못했다면 다음 프레임이미지를 읽는다

        if not ret :
            continue
        else :
            ret1, buffer = cv2.imencode(".jpg", frame)
            # 전송하기 위해 바이트로 변환
            frame = buffer.tobytes()
            # 전송
            yield (b'--frame\r\n' b'Content-Type: image/jpeg\r\n\r\n' + frame + b'\r\n')

            
from flask import Flask, render_template, Response
app = Flask(__name__)
# 초기화면
@app.route("/")
def index() :
    return render_template("index.html")

@app.route("/video_feed")
def video_feed() :
    return Response(get_frames(),
                   mimetype='multipart/x-mixed-replace; boundary=frame')

if __name__ == "__main__" :
    app.run(host="127.0.0.1", port=9000)

In [1]:
import cv2

# 9 X 9
block_size = 21
# 결과값에서 빼는 수치
C = 8

cap = cv2.VideoCapture(0)
# 이미지를 읽어서 한프레임씩 바로 전송하는 스트리밍 함수
def get_frames() :
    while True :
        ret, frame = cap.read()
        # 현재 프레임이미지를 읽지 못했다면 다음 프레임이미지를 읽는다

        if not ret :
            continue
        else :
            ret1, buffer = cv2.imencode(".jpg", frame)
            # 전송하기 위해 바이트로 변환
            frame = buffer.tobytes()
            # 전송
            yield (b'--frame\r\n' b'Content-Type: image/jpeg\r\n\r\n' + frame + b'\r\n')
            
def get_frames2() :
    while True :
        ret, frame = cap.read()
        frame=cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)
        frame = cv2.adaptiveThreshold(frame, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                               cv2.THRESH_BINARY, block_size, C)
        # 현재 프레임이미지를 읽지 못했다면 다음 프레임이미지를 읽는다

        if not ret :
            continue
        else :
            ret1, buffer = cv2.imencode(".jpg", frame)
            # 전송하기 위해 바이트로 변환
            frame = buffer.tobytes()
            # 전송
            yield (b'--frame\r\n' b'Content-Type: image/jpeg\r\n\r\n' + frame + b'\r\n')
            
from flask import Flask, render_template, Response
app = Flask(__name__)
# 초기화면
@app.route("/")
def index() :
    return render_template("index.html")

@app.route("/video_feed")
def video_feed() :
    return Response(get_frames(),
                   mimetype='multipart/x-mixed-replace; boundary=frame')

@app.route("/video_feed2")
def video_feed2() :
    return Response(get_frames2(),
                   mimetype='multipart/x-mixed-replace; boundary=frame')

if __name__ == "__main__" :
    app.run(host="127.0.0.1", port=9000)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:9000
Press CTRL+C to quit
127.0.0.1 - - [19/Sep/2022 15:22:11] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [19/Sep/2022 15:22:11] "GET /video_feed HTTP/1.1" 200 -
127.0.0.1 - - [19/Sep/2022 15:22:11] "GET /video_feed2 HTTP/1.1" 200 -
